<a href="https://colab.research.google.com/github/ccloveak/Study_Notes/blob/master/colab_control_google_drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 本地文件系统

## 上传文件
files.upload 返回一个上传文件的dict。dict的key是文件名，value是上传的数据。

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn,length=len(uploaded[fn])))

Saving 111.ai to 111.ai
User uploaded file "111.ai" with length 2337062 bytes


## 下载文件
files.download  调用浏览器下载文件

In [0]:
from google.colab import files

#with open('example.txt', 'w') as f:
#  f.write('some content')

files.download('111.ai')

# Google Drive

通过多种方式访问云端硬盘中的文件

*   Drive  REST API
*   使用整合的API包：PyDrive
*   本地控制 Google Drive

## 本地安装Google云端硬盘

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
#运行获取网址，点击网址，登陆自己的google账号，允许授权，获得授权码。

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
with open('/content/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/gdrive/My\ Drive/foo.txt

Hello Google Drive!

## PyDrive

示例
1. 身份验证
2. 文件上载
3. 文件下载  

[PyDrive文档](https://gsuitedevs.github.io/PyDrive/docs/build/html/index.html)

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. 验证并创建 PyDrive客户端
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive 参考:
# https://gsuitedevs.github.io/PyDrive/docs/build/html/index.html

# 2. 创建并上传文本文件
uploaded = drive.CreateFile({'title': 'Sample upload.txt'})
uploaded.SetContentString('Sample upload file こんにちは')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

# 3. 按id下载文件并打印内容
downloaded = drive.CreateFile({'id': uploaded.get('id')})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

     |████████████████████████████████| 993kB 2.8MB/s 
Uploaded file with ID 12Mf27iGzJbDQ84dcPKFn_Q4QQb1IZ51Z
Downloaded content "Sample upload file こんにちは"


## Drive REST API

 进行身份验证

In [0]:
from google.colab import auth
auth.authenticate_user()

构建一个Drive API客户端

In [0]:
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

### 使用Python在google drive 新建文件

In [0]:
# 创建本地文件
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file おはよう')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file おはよう

In [0]:
# 上传文件到google drive。
#
# https://developers.google.com/drive/v3/reference/files/create
# https://developers.google.com/drive/v3/web/manage-uploads
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': 'Sample file',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('/tmp/to_upload.txt', 
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

File ID: 1VOEvWi6UMwx64WV8c_GMpFbQG1m83jY0


###使用Python从Google Drive下载文件

In [0]:
# 下载刚刚上传的文件.
#
# Replace the assignment below with your file ID
# to download a different file.
#
# A file ID looks like: 1uBtlaggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1VOEvWi6UMwx64WV8c_GMpFbQG1m83jY0'

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

downloaded.seek(0)
print('Downloaded file contents are: {}'.format(downloaded.read()))

Downloaded file contents are: b'my sample file \xe3\x81\x8a\xe3\x81\xaf\xe3\x82\x88\xe3\x81\x86'


# Google Sheets

示例：使用现有的开源[gspread](https://github.com/burnash/gspread)库与Sheets进行交互

首先用pip安装gspread

In [0]:
!pip install --upgrade -q gspread

接下来，导入库，进行身份验证并创建表单界面。

In [0]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

gspread示例   （其他的访问[github](https://github.com/burnash/gspread#more-examples)）

##使用Python创建新表格

In [0]:
sh = gc.create('A new spreadsheet')

执行上述单元格后，新的电子表格将显示在sheets.google.com上的工作表列表中

In [0]:
# Open our new sheet and add some data.
worksheet = gc.open('A new spreadsheet').sheet1

cell_list = worksheet.range('A1:C2')

import random
for cell in cell_list:
  cell.value = random.randint(1, 10)

worksheet.update_cells(cell_list)

{'spreadsheetId': '1gFqkMTmqV7KYuBFN3QgPk0Bm_YkIJ2-r7hkVCRcf4Jg',
 'updatedCells': 6,
 'updatedColumns': 3,
 'updatedRange': "'工作表1'!A1:C2",
 'updatedRows': 2}

执行上面的单元格后，工作表将填充指定范围内的随机数

##使用Python下载从工作表下载数据并存为Pandas DataFrame

In [0]:
# Open our new sheet and read some data.
worksheet = gc.open('A new spreadsheet').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
pd.DataFrame.from_records(rows)

[['5', '8', '9'], ['1', '5', '8']]


,0,1,2
0,5,8,9
1,1,5,8
